# <b>Mục tiêu</b>
- Thu thập dữ liệu về thời tiết tại TP HCM theo từng ngày từ năm 2010 đến năm 2021.
<br><br>
- Các trường dữ liệu sẽ thu thập bao gồm:
    + Weather Type: Loại thời tiết
    + Average Temperature: Nhiệt độ trung bình trong ngày ($^\circ C$)
    + Highest Temperature: Nhiệt độ cao nhất trong ngày ($^\circ C$)
    + Lowest Temperature: Nhiệt độ thấp nhất trong ngày ($^\circ C$)
    + Wind Speed: Tốc độ gió (km/h)
    + Rain: Lượng mưa (mm)
    + Humidity: Độ ẩm (%)
    + Cloud: Độ che phủ của mây (%)
    + Pressure: Áp suất không khí (mb)
<br><br>
- Trả về:
    + hcm_weather_info.csv: File CSV chứa dữ liệu.
    + feature_description.csv: File CSV mô tả thông tin các trường dữ liệu.

# <b>Import</b>

In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup

In [4]:
%pip install Selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# <b>Thu thập dữ liệu</b>

In [4]:
url = 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx'
days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [5]:
# Thiết lập ChromeDriver
service = ChromeService(executable_path= ChromeDriverManager().install())
options = Options()
options.add_argument('--headless')                              # Ẩn màn hình Chrome, chỉ chạy dưới nền
options.add_argument('--window-size=1920, 1200')
driver = webdriver.Chrome(service= service, options= options)    # Khởi tạo Driver

driver.get(url)

weather_info_list = []              # List chứa dữ liệu thời tiết TP HCM 2010 - 2021
exception_occured = False

for i, year in enumerate(years):
    for month in range(12):
        for date in range(days[month]):
            try:
                # Nhập ngày cần tìm thông tin thời tiết
                search_date = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_MainContentHolder_txtPastDate"]'))
                )
                driver.execute_script(f'arguments[0].setAttribute("value", "{year}-{month + 1:02}-{date + 1:02}");', search_date)      # Eg: 2010-02-01 

                # Nhấn nút 'Get Weather' để hiện thị dữ liệu thời tiết
                get_weather_button =  WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="ctl00_MainContentHolder_butShowPastWeather"]'))
                )
                driver.execute_script('arguments[0].click();', get_weather_button)
                
                # Lấy nhiệt độ các mốc trong ngày để tính nhiệt độ trung bình
                temperature_info = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[4]/div[1]/div/div[3]/table'))
                )

                total_temperature = 0
                for k in range (3, 11):
                    temperature = temperature_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[4]/div[1]/div/div[3]/table/tbody/tr[{k}]/td[1]/p[2]')
                    total_temperature = total_temperature + int(re.findall(r'\d+', temperature.text)[0])
                average_temp_info = np.round(total_temperature / 8, 1)          

                # Các trường dữ liệu còn lại
                weather_info = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]'))
                )

                # Năm
                year_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}1]')

                # Loại thời tiết - Weather Type
                weather_type_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}2]/img')

                # Nhiệt độ cao nhất - Highest Temperature
                highest_temp_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}3]')

                # Nhiệt độ thấp nhất - Lowest Temperature
                lowest_temp_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}4]')

                # Tốc độ gió - Wind Speed
                wind_speed_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}5]')

                # Lượng mưa - Rain
                rain_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}6]')

                # Độ ẩm - Humidity
                humidity_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}7]')

                # Độ che phủ của mây - Cloud
                cloud_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}8]')

                # Áp suất không khí - Pressure
                pressure_info = weather_info.find_element(by= By.XPATH, value= f'//*[@id="aspnetForm"]/section[2]/div/div/div[1]/div[6]/div[2]/div[{i + 2}9]')
                
                # Lưu dữ liệu vào dictionary
                weather_in_date = {
                    'Date': f'{int(year_info.text)}-{month + 1:02}-{date + 1:02}',
                    'Weather Type': weather_type_info.get_attribute("title"),
                    'Average Temperature': average_temp_info,
                    'Highest Temperature': re.findall(r'\d+', highest_temp_info.text)[0],
                    'Lowest Temperature': re.findall(r'\d+', lowest_temp_info.text)[0],
                    'Wind Speed': re.findall(r'\d+', wind_speed_info.text)[0],
                    'Rain': re.findall(r'\d+.\d+', rain_info.text)[0],
                    'Humidity': re.findall(r'\d+', humidity_info.text)[0],
                    'Cloud': re.findall(r'\d+', cloud_info.text)[0],
                    'Pressure': re.findall(r'\d+', pressure_info.text)[0],
                    }
                        
                # Thêm thông tin lấy được vào list
                weather_info_list.append(weather_in_date)

            except Exception as e:
                print('An exception has occurred: ', e)
                exception_occured = True
                break  

        if exception_occured: break
    if exception_occured: break  
    
driver.quit()

In [6]:
hcm_weather_info_df = pd.DataFrame(weather_info_list) 
hcm_weather_info_df

,Date,Weather Type,Average Temperature,Highest Temperature,Lowest Temperature,Wind Speed,Rain,Humidity,Cloud,Pressure
0,2010-01-01,Partly cloudy,28.4,34,24,6,0.0,59,20,1010
1,2010-01-02,Cloudy,27.0,32,24,7,0.0,61,48,1010
2,2010-01-03,Partly cloudy,28.1,33,26,7,0.0,65,37,1010
3,2010-01-04,Partly cloudy,28.2,35,25,9,0.1,61,42,1009
4,2010-01-05,Partly cloudy,28.8,35,26,10,0.0,57,35,1009
...,...,...,...,...,...,...,...,...,...,...
4375,2021-12-27,Light rain shower,27.2,34,23,5,1.8,73,61,1012
4376,2021-12-28,Patchy rain possible,27.2,33,24,8,0.6,72,56,1012
4377,2021-12-29,Light rain shower,26.0,32,23,8,2.2,79,64,1012
4378,2021-12-30,Cloudy,24.5,28,22,7,0.0,79,45,1013


In [8]:
feature_description_list = [
    {'Feature': 'Date', 'Description': 'Ngày lấy dữ liệu (yyyy-mm-dd)'},
    {'Feature': 'Weather Type', 'Description': 'Loại thời tiết'},
    {'Feature': 'Average Temperature', 'Description': u'Nhiệt độ trung bình trong ngày (\N{DEGREE SIGN}C)'},
    {'Feature': 'Highest Temperature', 'Description': u'Nhiệt độ cao nhất trong ngày (\N{DEGREE SIGN}C)'},
    {'Feature': 'Lowest Temperature', 'Description': u'Nhiệt độ thấp nhất trong ngày (\N{DEGREE SIGN}C)'},
    {'Feature': 'Wind Speed', 'Description': 'Tốc độ gió (km/h)'},
    {'Feature': 'Rain', 'Description': 'Lượng mưa (mm)'},
    {'Feature': 'Humidity', 'Description': 'Độ ẩm (%)'},
    {'Feature': 'Cloud', 'Description': 'Độ che phủ của mây (%)'},
    {'Feature': 'Pressure', 'Description': 'Áp suất không khí (mb)'},
]

feature_description_df = pd.DataFrame(feature_description_list)
feature_description_df

,Feature,Description
0,Date,Ngày lấy dữ liệu (yyyy-mm-dd)
1,Weather Type,Loại thời tiết
2,Average Temperature,Nhiệt độ trung bình trong ngày (°C)
3,Highest Temperature,Nhiệt độ cao nhất trong ngày (°C)
4,Lowest Temperature,Nhiệt độ thấp nhất trong ngày (°C)
5,Wind Speed,Tốc độ gió (km/h)
6,Rain,Lượng mưa (mm)
7,Humidity,Độ ẩm (%)
8,Cloud,Độ che phủ của mây (%)
9,Pressure,Áp suất không khí (mb)


# <b>Lưu dữ liệu</b>

In [9]:
# Tạo thư mục lưu dữ liệu
list_file = os.listdir()
if 'Data' not in list_file:
     os.makedirs("Data")

In [10]:
# Lưu dữ liệu vào file CSV
hcm_weather_info_df.to_csv('./Data/hcm_weather_info.csv', index= False)
feature_description_df.to_csv('./Data/feature_description.csv', index= False)